In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem.rdmolops import RemoveStereochemistry

pd.options.mode.chained_assignment = None

# Directory Setup

In [2]:
raw_data_directory = '/Users/Avi/Dissertation/Data/Curated/Raw'
preprocessed_data_directory = '/Users/Avi/Dissertation/Data/Curated/Preprocessed'
os.makedirs(preprocessed_data_directory, exist_ok=True)
base_plot_directory = '/Users/Avi/Dissertation/Results/Exploratory_Analysis/Preprocessed_Data/Curated'
os.makedirs(base_plot_directory, exist_ok=True)

# Defining Functions

In [3]:
def get_top_files(directory, n=5):
    """Identify the top n files with the most compounds."""
    file_compound_counts = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv.gz'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            num_compounds = df['compound_chembl_id'].nunique()
            file_compound_counts.append((filename, num_compounds))
    
    file_compound_counts.sort(key=lambda x: x[1], reverse=True)
    return file_compound_counts[:n]

def clean_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        remover = SaltRemover()
        mol = remover.StripMol(mol)
        if mol is not None:
            RemoveStereochemistry(mol)
            clean_smiles = Chem.MolToSmiles(mol, canonical=True)
            return clean_smiles
    return None

def lipinski(df, smiles_column='canonical_smiles'):
    moldata = []
    for elem in df[smiles_column]:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
       
    baseData = []
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        row = [desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors]
        baseData.append(row)
    
    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames, index=df.index)
    
    return pd.concat([df, descriptors], axis=1)

def apply_lipinski(df):
    df['MW_violation'] = df['MW'] > 500
    df['LogP_violation'] = df['LogP'] > 5
    df['NumHDonors_violation'] = df['NumHDonors'] > 5
    df['NumHAcceptors_violation'] = df['NumHAcceptors'] > 10

    df['NumViolations'] = (df['MW_violation'].astype(int) +
                           df['LogP_violation'].astype(int) +
                           df['NumHDonors_violation'].astype(int) +
                           df['NumHAcceptors_violation'].astype(int))

    df_preprocessed = df[df['NumViolations'] < 2]
    return df_preprocessed[['assay_chembl_id', 'compound_chembl_id', 'pchembl_value', 'activity', 'canonical_smiles']]

def process_target(file_path, threshold=6.0):
    """Process a single target file."""
    df = pd.read_csv(file_path)
    df = df.drop_duplicates(subset='compound_chembl_id')
    df['activity'] = df['pchembl_value'].apply(lambda x: 1 if x >= threshold else 0)
    df['canonical_smiles'] = df['canonical_smiles'].apply(clean_smiles)
    df = df.dropna(subset=['canonical_smiles'])
    df = df.drop_duplicates(subset='canonical_smiles')
    df = lipinski(df)
    df = apply_lipinski(df)
    return df

# Main processing

In [4]:
print("Identifying Targets With Most Compounds")
top_five_files = get_top_files(raw_data_directory)
for file, count in top_five_files:
    print(f"{file}: {count} compounds")

print("\nProcessing Data")
target_data = {}
for filename, _ in top_five_files:
    target_id = filename.split('-')[0].split('_')[1]
    file_path = os.path.join(raw_data_directory, filename)
    target_data[target_id] = process_target(file_path)
    
    initial_count = pd.read_csv(file_path)['compound_chembl_id'].nunique()
    final_count = target_data[target_id]['compound_chembl_id'].nunique()
    
    print(f"\nTarget {target_id}:")
    print(f"Initial count: {initial_count} compounds")
    print(f"Final count: {final_count} compounds after preprocessing")
    
    output_file = f'Target_{target_id}_Curated_Preprocessed.csv'
    target_data[target_id].to_csv(os.path.join(preprocessed_data_directory, output_file), index=False)

print("\nPreprocessing complete.")

Identifying Targets With Most Compounds
target_CHEMBL4078-3.csv.gz: 4558 compounds
target_CHEMBL279-13.csv.gz: 3922 compounds
target_CHEMBL5763-3.csv.gz: 3355 compounds
target_CHEMBL240-15.csv.gz: 3139 compounds
target_CHEMBL4005-20.csv.gz: 3005 compounds

Processing Data

Target CHEMBL4078:
Initial count: 4558 compounds
Final count: 3717 compounds after preprocessing

Target CHEMBL279:
Initial count: 3922 compounds
Final count: 3268 compounds after preprocessing

Target CHEMBL5763:
Initial count: 3355 compounds
Final count: 2674 compounds after preprocessing

Target CHEMBL240:
Initial count: 3139 compounds
Final count: 2715 compounds after preprocessing

Target CHEMBL4005:
Initial count: 3005 compounds
Final count: 2734 compounds after preprocessing

Preprocessing complete.


# Exploratory Analysis of Target Variables After Preprocessing

In [5]:
print("\nExploratory Data Analysis")

for target_id, df in target_data.items():
    print(f"\nExploratory Data Analysis for Target {target_id}")

    target_plot_directory = base_plot_directory
    os.makedirs(target_plot_directory, exist_ok=True)

    sns.countplot(x='activity', data=df)
    plt.title(f'Activity Class Distribution for Target {target_id} (Curated)')
    plt.xlabel('Activity Class (0 = Inactive, 1 = Active)')
    plt.ylabel('Count')
    plt.savefig(os.path.join(target_plot_directory, f'{target_id}_Curated_Activity_Class_Distribution.png'))
    plt.clf()

    sns.boxplot(x='pchembl_value', data=df)
    plt.title(f'Box Plot of pChEMBL Values for Target {target_id}  (Curated)')
    plt.xlabel('pChEMBL Value')
    plt.savefig(os.path.join(target_plot_directory, f'{target_id}_Curated_pChEMBL_Box_Plot.png'))
    plt.clf()

    sns.histplot(df['pchembl_value'], kde=True)
    plt.title(f'Distribution of pChEMBL Values for Target {target_id}  (Curated)')
    plt.xlabel('pChEMBL Value')
    plt.ylabel('Frequency')
    plt.savefig(os.path.join(target_plot_directory, f'{target_id}_Curated_pChEMBL_Distribution.png'))
    plt.clf()

print("Exploratory Data Analysis complete.")


Exploratory Data Analysis

Exploratory Data Analysis for Target CHEMBL4078

Exploratory Data Analysis for Target CHEMBL279

Exploratory Data Analysis for Target CHEMBL5763

Exploratory Data Analysis for Target CHEMBL240

Exploratory Data Analysis for Target CHEMBL4005
Exploratory Data Analysis complete.


<Figure size 640x480 with 0 Axes>